# Load Packages

What we can recall from critisicm regarding this presentation:

- did not take the equal weighted as benchmark
- use total return indices (I think)

In [1]:
#### Import required Packages ####
import numpy as np
import pandas as pd
import seaborn as sb
import datetime as dt
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd

In [2]:
# Seaborn
sb.set_style("ticks")
sb.mpl.rc("figure", figsize=(16,8))
sb.mpl.rc("font", size=14)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

# Load Data

In [3]:
# rawdata is direct import from excel

# Import of excel data, sheet by sheet
xls_ETF = pd.ExcelFile("1_Data/Data_ETF_YIELDS_1.0.xlsx")
xls_inflation = pd.ExcelFile("1_Data/Data_INFLATION_1.0.xlsm")

ETF_raw = pd.DataFrame(pd.read_excel(xls_ETF, 1))
Yield_raw = pd.DataFrame(pd.read_excel(xls_ETF, 2))

cpicore_raw = pd.DataFrame(pd.read_excel(xls_inflation, 0))
targetrates_raw = pd.DataFrame(pd.read_excel(xls_inflation, 1))
#rawdata.columns = ["Trading_day", "Date", "MSCI_EU", "Healthcare", "Finacials", "Consumer_staples","Industrials","Consumer_discretionary","Materials", "Information_technology", "Energy", "Utilities", "Communication_services","Real_estate"]


# keep a safe copy of the rawdata to compare the changes
ETF = ETF_raw
Yield = Yield_raw
cpicore = cpicore_raw
targetrates = targetrates_raw

In [4]:
# Drop all the columns in which only text is and safe them separately --> ONLY RUN ONCE
ETF_text = ETF.iloc[0,:]
Yield_text = Yield.iloc[0,:]
cpicore_text = cpicore.iloc[0:1,:]

ETF.drop([0], inplace = True)
Yield.drop([0], inplace = True)
cpicore.drop([0,1], inplace = True)

# Set date as index
ETF.set_index("Dates", inplace=True)
Yield.set_index("Dates", inplace=True)
cpicore.set_index("Dates", inplace=True)
targetrates.set_index("Dates", inplace=True)

# Value

Recent infl – the average of the headline and core annual CPI inflation rate
--> we don't have headline inflation


Effective target infl - The effective inflation target is the mean of the target range announced or implied by the authorities plus an adjusted for past “target misses”, which is the last 3 years’ average gap between actual inflation and the target means
--> we don't have target rates for all countries --> use 2.5 as target rate

Formula: (1/n)*recent infl + ((n-1)/n)*effective target infl

In [5]:
# create the rolling 3-year average of the core inflation

# first I have to set up an empty dictionary to store the rolling averages
cpicore_avg = {}
for i in cpicore.columns:
    cpicore_avg[i] = []

# next up we iterate over the cpicore data to get the index and safe them in the dictionary
indices_reverse = cpicore.index[::-1]
runner = len(indices_reverse)
for i in indices_reverse:
    if runner == 35:
        break
    
    for j,k in enumerate(cpicore.columns):
        inflation = np.nanmean(cpicore.iloc[runner-36:runner, j])
        cpicore_avg[k].append(inflation)
    runner -= 1
    
#cpicore_avg
cpicore_avg_trimmed = pd.DataFrame(cpicore_avg)

<ipython-input-5-526646c4995a>:16: RuntimeWarning: Mean of empty slice
  inflation = np.nanmean(cpicore.iloc[runner-36:runner, j])


In [6]:
# now calculate the difference between 3-year average and target inflation

eff_target_inf = dict(cpicore_avg)

# special cases (Australia, Canada, UK, Japan, US, Switzerland) for which we have an actual target
for key in cpicore_avg.keys():
    if key == "Australia":
        for i,k in enumerate(cpicore_avg[key]):
            if k >= 2.0 and k <= 3.0:
                eff_target_inf[key][i] = 0
            elif k < 2.0:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 2
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 3
                
    elif key == "Canada":
        for i,k in enumerate(cpicore_avg[key]):
            if k >= 1.0 and k <= 3.0:
                eff_target_inf[key][i] = 0
            elif k < 2.0:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 1
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 3
                
    elif key == "United Kingdom":
        for i,k in enumerate(cpicore_avg[key]):
            if k == 2.0:
                eff_target_inf[key][i] = 0
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 2
                
    elif key == "Japan":
        for i,k in enumerate(cpicore_avg[key]):
            if k == 3.3:
                eff_target_inf[key][i] = 0
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 3.3
                
    elif key == "United States":
        for i,k in enumerate(cpicore_avg[key]):
            if k == 2.0:
                eff_target_inf[key][i] = 0
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 2
                
    elif key == "Switzerland":
        for i,k in enumerate(cpicore_avg[key]):
            if k >= 0.0 and k <= 2.0:
                eff_target_inf[key][i] = 0
            elif k < 0.0:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 0
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 2
                
    else:
        for i,k in enumerate(cpicore_avg[key]):
            if k == 2.5:
                eff_target_inf[key][i] = 0
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 2.5


In [35]:
# now we calculate the value formula:
# (1/n)*recent inflation + ((n-1)/n)*effective target inflation for the years: 2, 5, 10, 20, 30

# first we need to align the dataframes 
cpicore_trimmed = cpicore[35:]
#cpicore_avg_trimmed = pd.DataFrame(cpicore_avg)
cpicore_avg_trimmed.set_index(cpicore_trimmed.index, inplace = True)

# second we create an empty dictionary to fill with the calculations
formula_values = {}
x = 0
for i in cpicore_trimmed.columns:
    x += 1
    for j in [2,5,10,20,30]:
        formula_values[f"{x},{j}"] = []
#formula_values

# now that we have the empty dictionary to fill, we start with caluclating the inflation expectation
keys = list(formula_values.keys())

runner = 0
for i,k in enumerate (cpicore_trimmed.columns):
    for j in [2,5,10,20,30]:
        formula_values[keys[runner]].append((1/j)*cpicore_trimmed[k] +((j-1)/j)*cpicore_avg_trimmed[k])
        runner += 1

In [62]:
# in a next step we adjust the yield data to have the same lookback as the other data

Yield_trimmed = Yield[35:]

Real_yield = pd.DataFrame(columns = Yield_trimmed.columns, index = Yield_trimmed.index)

# here we calculate the difference between the yield etf and the inflation
for i,k in enumerate(Real_yield.columns):
    key = keys[i]
    for j,l in enumerate(Real_yield.index):
        Real_yield.iloc[j,i] = Yield_trimmed.iloc[j,i] - formula_values[key][0][j]

In [63]:
Real_yield

,Germany,Germany.1,Germany.2,Germany.3,Germany.4,France,France.1,France.2,France.3,France.4,...,Singapore,Singapore.1,Singapore.2,Singapore.3,Singapore.4,India,India.1,India.2,India.3,India.4
Dates,,,,,,,,,,,,,,,,,,,,,
2002-11-29,-0.294444,0.288889,0.94,1.460556,1.494074,-0.990833,-0.233333,0.5725,1.035417,1.133056,...,0.153889,0.772222,1.8238,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-12-31,-0.445667,0.068333,0.824,1.399333,1.427444,-1.048111,-0.426778,0.442,0.941889,1.040519,...,0.081111,0.697778,1.4921,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-01-31,-0.487333,0.072667,0.842,1.419667,1.421556,-1.501444,-0.549111,0.308,0.834056,0.965741,...,0.099722,0.635556,1.4752,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-02-28,-0.705,-0.005,0.838,1.534,1.597667,-1.830833,-0.784333,0.2045,0.876417,1.051056,...,-0.037222,0.534444,1.3422,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-03-31,-0.709667,0.233333,1.074,1.795333,1.856444,-1.429889,-0.422222,0.529,1.122611,1.288815,...,-0.191111,0.562222,1.1801,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,-3.502944,-2.696111,-2.3675,-2.181694,-2.253093,-2.525389,-1.269222,-0.6665,-0.282139,-0.228019,...,1.131667,1.529667,1.8573,1.809167,1.716222,NaN,NaN,NaN,NaN,NaN
2022-12-30,-3.111,-2.142,-1.764,-1.5315,-1.533333,-1.628722,-0.482556,0.1185,0.474528,0.58187,...,1.204889,1.450222,1.8881,1.487889,1.375519,NaN,NaN,NaN,NaN,NaN
2023-01-31,-3.210111,-2.387778,-2.024,-1.783111,-1.827481,-1.521944,-0.628111,-0.1495,0.269306,0.323907,...,0.952167,1.357667,1.699,1.690417,1.512056,NaN,NaN,NaN,NaN,NaN


# Momentum

# Cut Momentum Frame

In [ ]:
# Exclude Trading Days column and MSCI_EU Benchmark to have only Momentum columns
prices = df.loc[:,"Healthcare":"Real_estate"]
prices.dtypes

In [ ]:
# Calculating Monthly returns
mtl_ret = prices.pct_change().resample("M").agg(lambda x: (1+x).prod()-1) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
mtl_ret

In [ ]:
past_11 = (mtl_ret+1).rolling(11).apply(np.prod)-1 # accumulate returns over 11 months 
past_11.head(11)

In [ ]:
# Defining formation date <-  where portfolio gets created
formation = dt.datetime(1999, 12, 31, 0, 0)

In [ ]:
# Defining MeasurementDate <- up to last date of month before formation date
end_measurement = formation - MonthEnd(1)
end_measurement

In [ ]:
# Past 12 Month performance without most recent month <- in this case Jan, Feb, Mar, Apr, Jun, Jul, Sep, Oct, Nov but not December!
ret_12 = past_11.loc[end_measurement]
ret_12 = ret_12.reset_index()
ret_12.rename(columns={ ret_12.columns[1]: "returns" }, inplace = True)
ret_12

In [ ]:
ret_12 = ret_12.sort_values(["returns"], ascending=False)
ret_12["bucket"] = [2,2,2,2,2,1,0,0,0,0,0] # Make three buckets
ret_12

In [ ]:
# Create winners and loser list based on bucket
winners = ret_12[ret_12.bucket == 2]
losers = ret_12[ret_12.bucket == 0]
winners

In [ ]:
# Calculate the winner and loser returns over the last 12 months excluding the most recent month
winnerret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(winners["index"])]
loserret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(losers["index"])]

In [ ]:
Momentumprofit = winnerret.mean() - loserret.mean()
Momentumprofit

# Functionalising this Methodology

In [ ]:
def data_cleaner(pandas_df, index="Date", prices_start='Healthcare', prices_end='Real_estate', resample_freq='M', lookback=11):
    print(pandas_df.isnull().values.any())
    df = pandas_df.set_index(index, inplace=False)
    prices = df.loc[:,prices_start:prices_end]
    mtl_ret = prices.pct_change().resample(resample_freq).agg(lambda x: (1+x).prod()-1)
    past_11 = (mtl_ret+1).rolling(lookback).apply(np.prod)-1
    return prices, mtl_ret, past_11


In [ ]:
dfff = dff.set_index("Date", inplace = False)
x = dfff.resample("M")
type(x)

In [ ]:
#test out the data_cleaner function

prices, mtl_ret, past_11 = data_cleaner(dff, lookback = 3, resample_freq= 'BM')
print(f'\nprices:\n {prices}, \nmonthly_return:\n {mtl_ret}, \npast_11:\n {past_11}')

In [ ]:
past_11.head(5)

In [ ]:
def momentum(formation, past_11, num_l_s=5):
    end_measurement = formation - MonthEnd(1)
    ret_12 = past_11.loc[end_measurement]
    ret_12 = ret_12.reset_index()
    ret_12.rename(columns={ ret_12.columns[1]: "returns" }, inplace = True)
    ret_12 = ret_12.sort_values(["returns"], ascending=False)
    
    middle = 11-2*num_l_s
    l_s = []
    for i in range(num_l_s):
        l_s.append(2)
    for i in range(middle):
        l_s.append(1)
    for i in range(num_l_s):
        l_s.append(0)
    ret_12["bucket"] = l_s # Make arbitrary number of buckets
    
    winners = ret_12[ret_12.bucket == 2]
    losers = ret_12[ret_12.bucket == 0]
    winnerret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(winners["index"])]
    loserret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(losers["index"])]
    Momentumprofit = winnerret.mean() - loserret.mean()
    return Momentumprofit







In [ ]:
formation = dt.datetime(1999, 12, 31, 0, 0)
momentum(formation, past_11) # has to be identical with [35]

In [ ]:
def benchmarker(pandas_df, profits_list):
    dff = pandas_df.set_index("Date", inplace=False)
    
    benchmark = dff[["MSCI_EU"]]
    benchmark = benchmark.loc["1999-12-31 00:00:00":"2022-09-30 00:00:00"]
    
    bm_mtl_ret = mtl_ret = benchmark.pct_change().resample("M").agg(lambda x: (1+x).prod()-1) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
    comparison = pd.DataFrame(bm_mtl_ret)
    comparison["mom_ret"] = profits_list
    comparison.rename(columns={ comparison.columns[0]: "bench_ret" }, inplace = True)
    comparison["bench_indexed"] = comparison["bench_ret"].add(1).cumprod()
    comparison["mom_indexed"] = comparison["mom_ret"].add(1).cumprod()
    print(comparison)
    
    fig, ax = plt.subplots()
    ax.plot(comparison.index,comparison.mom_indexed, label="Momentum", color="red")
    ax.plot(comparison.index,comparison.bench_indexed, label="MSCI EU", color="blue")
    ax.legend(loc="best")
    ax.set_ylabel('Monthly Returns')
    ax.set_title("Momentum long/short 5 sectors monthly rebalancing vs MSCI EU, indexed 31.12.1999")


In [ ]:
# Creating loop for every month  starting at formation date -> creates df with all these dates
for i in range(273): # Eig wären es 273 Monate zwischen 31.12.1999 und 31.12.2022 -> stimmt irgendwie ned ganz aber lauft jetzt halt so bis endi august
    print(formation + MonthEnd(i))

#wären es nicht 276 Monate? has mit emene Zeitspannen-rechner usgrechnet --> gahd ets bis endi november 2022
for i in range(276):
    print(formation + MonthEnd(i))


In [ ]:
# Creating loop for storing the profits and the realized dates of the momentum strategy
profits = []
dates = []

'''
for i in range(273):
    profits.append(momentum(formation + MonthEnd(i)))
    dates.append(formation + MonthEnd(i))
'''

#version 2.0 (da gahds nume bis 274 --> gid en error wenn en monet meh nimmsch)
for i in range(274):
    profits.append(momentum(formation + MonthEnd(i), past_11))
    dates.append(formation + MonthEnd(i))


In [ ]:
benchmarker(df, profits)

In [ ]:
mom = pd.DataFrame({"Dates": dates,"Profits": profits})
mom

# Benchmarking

In [ ]:
benchmark = df[["MSCI_EU"]]
benchmark = benchmark.loc["1999-12-31 00:00:00":"2022-09-30 00:00:00"]
benchmark

In [ ]:
bm_mtl_ret = mtl_ret = benchmark.pct_change().resample("M").agg(lambda x: (1+x).prod()-1) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
bm_mtl_ret


In [ ]:
#compare momentum result vs benchmark
comparison = pd.DataFrame(bm_mtl_ret)
comparison["mom_ret"] = profits
comparison.rename(columns={ comparison.columns[0]: "bench_ret" }, inplace = True)
comparison["bench_indexed"] = comparison["bench_ret"].add(1).cumprod()
comparison["mom_indexed"] = comparison["mom_ret"].add(1).cumprod()
comparison


In [ ]:
comparison

# Comparison Plot

In [ ]:
# Plotting
fig, ax = plt.subplots()
ax.plot(comparison.index,comparison.mom_indexed, label="Momentum", color="red")
ax.plot(comparison.index,comparison.bench_indexed, label="MSCI EU", color="blue")
ax.legend(loc="best")
ax.set_ylabel('Monthly Returns')
ax.set_title("Momentum long/short 5 sectors monthly rebalancing vs MSCI EU, indexed 31.12.1999")